In [4]:
import nltk
import re
import unicodedata
from uszipcode import SearchEngine
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import datefinder
from datetime import datetime
from collections import defaultdict
import MySQLdb
import MySQLdb.cursors
import requests


class education_details():
    
    #def __init__(self):
    
    def search_keyword(self,text, keyword_list):
        for word in keyword_list:
            word = str(word)
            if word.title() in text or word.upper() in text or word.capitalize() in text:
                return True
        return False
    
    def load_education_segment(self,pdf_to_text_list,education_keywords,
                               project_keywords, work_experience_keywords,other_keywords,
                              education_segment):#skill_keywords,
        
        #print(pdf_to_text_list)
        
        def validate_text(text):
            
            user_segment = []

            # check email in name object
            if text.find("@") > 0 or text.find("github.com") >= 0 or text.find("linkedin.com") >= 0:
                user_segment.append(text)
                # print "Email found"
                return False

            # check phone number in name object
            cell = re.search("\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}",
                             text)
            if cell:
                # print "Cell found"
                user_segment.append(text)
                return False

            # check hyperlinks in name object
            urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
            if len(urls) > 0:
                # print "link found"
                user_segment.append(text)
                return False

            return True

        # Extract Education Segment
        #print(type(pdf_to_text_list))
        for i, text in enumerate(pdf_to_text_list):
            #print(i,"=====",text)
            text = text.replace("|","")
            #text = text.encode('ascii', 'ignore')
            flag = False
            if self.search_keyword(text, education_keywords):
                #print(text)
                education_segment.append(text)
                #print("===== stuck here? ======= ",education_segment)
                i += 1
                flag = True
                while True and i < len(pdf_to_text_list):
                    
                    #print("Stuck from here", i)
                    text = pdf_to_text_list[i]
                    #text = re.sub(r'[^\w\s]','',text)
                    text = text.replace("|","")
                    #text = text.encode('ascii', 'ignore')
                    #print("========in while loop =======", text)
                    if not self.search_keyword(text, work_experience_keywords) and not self.search_keyword(
                        text, project_keywords)  and not self.search_keyword(
                        text, other_keywords): #and not self.search_keyword(text, skill_keywords)
                        if validate_text(text):
                            education_segment.append(text)
                    else:
                        break
                    i += 1      
            if flag:
                break
                
        return education_segment
    
    "======== first Education extraction function ========"
    
    def extract_education_details(self,qualification_word_dict,
                                                 abbr_list,segment,edu_obj,infoDict):#university_word_dict,major_word_list,
        cnx = MySQLdb.connect(host = 'localhost',
                             database = 'cv_parse',
                             user = 'root',
                             password = 'root', 
                             compress = 1,
                             cursorclass=MySQLdb.cursors.DictCursor)

        cursor = cnx.cursor()
        education_segment = segment
        edu_obj = edu_obj
        
        infoDict = infoDict
        infoDict['edu_keyword'] = []
        
        for i in range(0,len(education_segment)):
            university_flag = False
            major_flag = False
            date_flag = False
            degree_flag = False
            #print(i)
            text = education_segment[i]
            #text = re.sub(r'[^\w\s]','',text)
            #print(text)

            #print(university_flag)
            if not university_flag:
                text = text.replace("'","") 
                query = """SELECT university FROM cv_parse.university_table
                        WHERE '"""+ text + """' LIKE CONCAT('%',university,'%')"""
                #print(query)
                cursor.execute(query)
                uni = cursor.fetchall()

                #print(uni)

                if uni:
                    university = uni[0]['university']
                    closest_university = university
                    university_flag = True

                    score = fuzz.ratio(str(university).lower(), text.lower())
                    if score > 90 and max_score < score and not text == " ":
                        max_score = score
                        closest_university_2 = university
                        university_flag = True

                if university_flag:
                    #print(closest_university)
                    edu_obj['university'].append(closest_university)
                    #a.append(edu_obj['university'])
                    #print(a)
                    #addr_list = university_word_dict.get(closest_university)
                    #print(addr_list)
                    #edu_obj['set_addr'].append([addr_list[0], addr_list[1], addr_list[2], addr_list[3]])

                    j = i
                    while(True):
                        max_len = 0
                        degree = ""

                        # Find Date
                        if not date_flag:
                            # Find date from Text
                            #print(text)
                            matches = list(datefinder.find_dates(text))
                            #print(matches)
                            edu_date = ""
                            if not matches == []:
                                # Current Year
                                currentYear = datetime.now().year
                                #print(currentYear)
                                for temp_date in matches:
                                    if not temp_date.year > currentYear + 4:
                                        edu_date += str(temp_date.month) + "/" + str(temp_date.year) + " - "
                                        #print(edu_date)
                                edu_obj['year'].append(edu_date[:(len(edu_date) - 2)])
                                date_flag = True

                        # Find Degree and Major
                        # major_flag = False
                        if not degree_flag:
                            for abbr, val in qualification_word_dict.items():
                                score1 = fuzz.ratio(str(abbr).lower(), text.lower())
                                score2 = fuzz.ratio(str(val).lower(), text.lower())
                                # IF match perfectly it score will come more than 90% so we can get perfect name
                                if score1 > 90 or score2 > 90 and max_len < len(val):
                                    max_len = len(val)
                                    degree = val
                                # Check in Abbr list if in it get name
                                if not major_flag and str(abbr).lower() in text.lower():
                                    edu_obj['major'].append(val)
                                    major_flag = True


                            # Extract Degree Name Loop
                            #for word in major_word_list:
                            text = text.replace("'","")
                            query = """SELECT major FROM cv_parse.major_table
                                    WHERE '"""+ text + """' LIKE CONCAT('%',major,'%')"""
                            cursor.execute(query)
                            major = cursor.fetchall()

                            if major:
                                word = major[0]['major'] 
                                score3 = fuzz.partial_ratio(str(word).lower(), text.lower())
                                # IF score more than 90 will give Categories of study Ex: Civil Engineering
                                if score3 > 90 and max_len < len(word):
                                    max_len = len(word)
                                    #print(word)
                                    degree = word

                                if not max_len == 0 and not degree_flag:
                                    edu_obj['degree'].append(degree)
                                    degree_flag = True

                        i += 1

                        if not i >= len(education_segment):
                            text = education_segment[i]
                        else:
                            print("Else statment")
                            # Iterate from 0 until the index at which university was found
                            k = 0
                            while k < j:
                                text = education_segment[k]

                                if not edu_obj['year']:# == "":
                                    matches = list(datefinder.find_dates(text))
                                    #print("Find date")
                                    edu_date = ""
                                    if not matches == []:
                                        currentYear = datetime.now().year
                                        for temp_date in matches:
                                            if not temp_date.year > currentYear + 4:
                                                edu_date += str(temp_date.month) + "/" + str(temp_date.year) + " - "
                                        edu_obj['year'].append(edu_date[:(len(edu_date) - 2)])

                                #date_flag = True

                                # Find Degree
                                if not edu_obj['degree']:# == "":
                                    for abbr, val in qualification_word_dict.items():
                                        score1 = fuzz.ratio(str(abbr).lower(), text.lower())
                                        score2 = fuzz.ratio(str(val).lower(), text.lower())
                                        if score1 > 90 or score2 > 90 and max_len < len(val):
                                            max_len = len(val)
                                            degree = val

                                    if not major_flag and str(abbr).lower() in text.lower():
                                        edu_obj['major'].append(val)
                                        #major_flag = True
        
                                    #for word in major_word_list:
                                    text = text.replace("'","")
                                    query = """SELECT major FROM cv_parse.major_table
                                            WHERE '"""+ text + """' LIKE CONCAT('%',major,'%')"""
                                    cursor.execute(query)
                                    major = cursor.fetchall()

                                    if major:
                                        word = major[0]['major'] 
                                        score3 = fuzz.partial_ratio(str(word).lower(), text.lower())
                                        if score3 > 90 and max_len < len(word):
                                            max_len = len(word)
                                            degree = word
                                        if not max_len == 0 and not degree_flag:
                                            edu_obj['degree'].append(degree)
                                            degree_flag = True
                                k+=1

                            break

                    else:
                        # Search before university name in the education segment
                        date_flag, degree_flag, major_flag = find_date_degree_major_gpa(text, date_flag, degree_flag, major_flag)
               
            if not edu_obj['university'] or not edu_obj['year'] or not edu_obj['degree'] or not edu_obj['major'] :
                #print(text, "Not parse")
                infoDict['edu_keyword'].append(text)
        
            #print("Not Parse =====", infoDict['keyword'])

        return edu_obj,infoDict
            
    
    "==== Final education extract function ====="
    
    def parse_education_segment(self,qualification_word_dict,
                                                 abbr_list,segment,edu_obj,infoDict):#university_word_dict,major_word_list,
        
        cnx = MySQLdb.connect(host = 'localhost',
                             database = 'cv_parse',
                             user = 'root',
                             password = 'root', 
                             compress = 1,
                             cursorclass=MySQLdb.cursors.DictCursor)

        cursor = cnx.cursor()
        education_segment = segment
        #print("In Extract Function", education_segment)
        
        edu_obj = edu_obj
        infoDict = infoDict
        infoDict['edu_keyword'] = []

        def find_date_degree_major_gpa(text, date_flag, degree_flag, major_flag):
            #print(text,date_flag, degree_flag, major_flag)
            max_len = 0
            # Find GPA
            index = text.find("GPA")
            #print(index)
            if index >= 0:

                edu_obj['gpa'] = (text[index + 4:]).strip()

            # Find Date
            if not date_flag:
                matches = list(datefinder.find_dates(text))
                edu_date = ""
                if not matches == []:
                    currentYear = datetime.now().year
                    for temp_date in matches:
                        if not temp_date.year > currentYear + 4:
                            edu_date += str(temp_date.month) + "/" + str(temp_date.year) + " - "
                    edu_obj['year'] = edu_date[:(len(edu_date) - 2)]
                    date_flag = True

            # Find Degree and Major
            if not degree_flag:
                for abbr, val in qualification_word_dict.items():
                    score1 = fuzz.ratio(str(abbr).lower(), text.lower())
                    score2 = fuzz.ratio(str(val).lower(), text.lower())
                    if score1 > 90 or score2 > 90 and max_len < len(val):
                        max_len = len(val)
                        degree = val

                    if not major_flag and str(abbr).lower() in text.lower():
                        edu_obj['major'] = val
                        major_flag = True
                        
                text = text.replace("'","")
                query = """SELECT major FROM cv_parse.major_table
                        WHERE '"""+ text + """' LIKE CONCAT('%',major,'%')"""
                cursor.execute(query)
                major = cursor.fetchall()

                if major:
                    word = major[0]['major'] 
                    score3 = fuzz.partial_ratio(str(word).lower(), text.lower())
                    if score3 > 90 and max_len < len(word):
                        max_len = len(word)
                        degree = word
                        edu_obj['degree'] = degree
                        date_flag = True


            return  date_flag, degree_flag, major_flag

        # Find indexes of all university
        max_score, longest_length = 0, 0
        closest_university_1, closest_university_2 = "", ""
        university_flag = False
        degree_flag = False
        date_flag = False
        major_flag = False
        #edu_obj = defaultdict(list)
    
        edu_obj['year'] = []
        edu_obj['university'] = []
        #edu_obj['set_addr'] = []
        edu_obj['degree'] = []
        edu_obj['major'] = []
        
        for i in range(0,len(education_segment)):
            university_flag = False
            major_flag = False
            date_flag = False
            degree_flag = False
            #print(i)
            text = education_segment[i]
            #print(text)

            #print(university_flag)
            if not university_flag:
                text = text.replace("'","")
                query = """SELECT university FROM cv_parse.university_table
                        WHERE '"""+ text + """' LIKE CONCAT('%',university,'%')"""
                #print(query)
                cursor.execute(query)
                uni = cursor.fetchall()

                print(uni)

                if uni:
                    university = uni[0]['university']
                    closest_university = university
                    university_flag = True

                    score = fuzz.ratio(str(university).lower(), text.lower())
                    #print (university, score)
                    if score > 90 and max_score < score and not text == " ":
                        #print(score, university, [text])
                        max_score = score
                        closest_university = university
                        university_flag = True
                        #print(closest_university_2)

                if university_flag:
                    edu_obj['university'].append(closest_university)
                    #a.append(edu_obj['university'])
                    #print(a)
                    #addr_list = university_word_dict.get(closest_university)
                    #print(addr_list)
                    #edu_obj['set_addr'].append((addr_list[0], addr_list[1], addr_list[2], addr_list[3]))

                    j = i
                    while(True):

                        max_len = 0
                        degree = ""

                        # Find Date
                        if not date_flag:
                            #print(text)
                            matches = re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})', text)
                            #print(matches)
                            if matches != []:
                                edu_obj['year'].append(matches)
                                date_flag = True
                            # Find date from Text
                            #print(text)

                            if edu_obj['year'] == '':
                                matches = list(datefinder.find_dates(text))
                                #print(matches)
                                edu_date = ""
                                if not matches == []:
                                    # Current Year
                                    currentYear = datetime.now().year
                                    #print(currentYear)
                                    for temp_date in matches:
                                        #print(temp_date)
                                        #print(currentYear + 4 )
                                        # Person not Studentthatsy calculate year from current year
                                        if not temp_date.year > currentYear + 4:
                                            edu_date += str(temp_date.month) + "/" + str(temp_date.year) + " - "
                                            #print(edu_date)
                                    edu_obj['year'].append(edu_date[:(len(edu_date) - 2)])
                                    date_flag = True
                                else:
                                    matches = re.findall(r'.*([1-3][0-9]{3})', text)
                                    #print(matches)
                                    if matches != []:
                                        edu_obj['year'].append(matches[0])
                                    date_flag = True

                        # Find Degree and Major
                        # major_flag = False
                        if not degree_flag:
                            for abbr, val in qualification_word_dict.items():
                                score1 = fuzz.ratio(str(abbr).lower(), text.lower())
                                score2 = fuzz.ratio(str(val).lower(), text.lower())
                                # IF match perfectly it score will come more than 90% so we can get perfect name
                                if score1 > 90 or score2 > 90 and max_len < len(val):
                                    #print(val,"===",max_len)
                                    max_len = len(val)
                                    degree = val
                                    #print(max_len, degree)
                                # Check in Abbr list if in it get name

                                if not major_flag and str(abbr).lower() in text.lower():
                                    edu_obj['major'].append(val)
                                    major_flag = True


                            # Extract Degree Name Loop
                            #for word in major_word_list:
                            text = text.replace("'","")
                            query = """SELECT major FROM cv_parse.major_table
                                        WHERE '"""+ text + """' LIKE CONCAT('%',major,'%')"""
                            cursor.execute(query)
                            major = cursor.fetchall()

                            if major:
                                word = major[0]['major']
                                score3 = fuzz.partial_ratio(str(word).lower(), text.lower())

                                # IF score more than 90 will give Categories of study Ex: Civil Engineering
                                if score3 > 90 and max_len < len(word) and text != " ":
                                    #print(word, "==", text, "==", score3,"==", max_len)
                                    max_len = len(word)
                                    degree = word

                                if not max_len == 0 and not degree_flag:
                                    #print(degree)
                                    edu_obj['degree'].append(degree)
                                    degree_flag = True


                        i += 1

                        if not i >= len(education_segment):
                            text = education_segment[i]
                        else:
                            print("Else statment")

                            # Iterate from 0 until the index at which university was found
                            k = 0
                            while k < j:
                                text = education_segment[k]
                                if edu_obj['year'] == "":
                                    matches = list(datefinder.find_dates(text))
                                    #print("matches")
                                    edu_date = ""
                                    if not matches == []:
                                        currentYear = datetime.now().year
                                        for temp_date in matches:
                                            if not temp_date.year > currentYear + 4:
                                                edu_date += str(temp_date.month) + "/" + str(temp_date.year) + " - "
                                        edu_obj['year'].append(edu_date[:(len(edu_date) - 2)])
                                        date_flag = True
                                    else:
                                        match = re.findall(r'.*([1-3][0-9]{3})', text)
                                        #print(match)
                                        if match != []:
                                            edu_obj['year'].append(match)
                                            date_flag = True

                                #date_flag = True

                                # Find Degree
                                if not edu_obj['degree']:# == "":
                                    for abbr, val in qualification_word_dict.items():
                                        score1 = fuzz.ratio(str(abbr).lower(), text.lower())
                                        score2 = fuzz.ratio(str(val).lower(), text.lower())
                                        if score1 > 90 or score2 > 90 and max_len < len(val):
                                            max_len = len(val)
                                            degree = val

                                    if not major_flag and str(abbr).lower() in text.lower():
                                        edu_obj['major'].append(val)
                                        #major_flag = True
                                        
                                    text = text.replace("'","")
                                    query = """SELECT major FROM cv_parse.major_table
                                                WHERE '"""+ text + """' LIKE CONCAT('%',major,'%')"""
                                    cursor.execute(query)
                                    major = cursor.fetchall()

                                    if major:
                                        word = major[0]['major']
                                        score3 = fuzz.partial_ratio(str(word).lower(), text.lower())
                                        if score3 > 90 and max_len < len(word):
                                            max_len = len(word)
                                            degree = word
                                k+=1

                            break

                    else:
                        # Search before university name in the education segment
                        date_flag, degree_flag, major_flag = find_date_degree_major_gpa(text, date_flag, degree_flag, major_flag)

            if not edu_obj['university'] or not edu_obj['year'] or not edu_obj['degree'] or not edu_obj['major'] :
                    #print(text, "Not parse")
                    infoDict['edu_keyword'].append(text)
        
            #print("Not Parse =====", infoDict['keyword'])
        #print(infoDict)
        return edu_obj,infoDict

        if not edu_obj['university']:
            print("Another Function")
            self.extract_user_detail(segment,
                                qualification_word_dict,abbr_list)#university_word_dict,major_word_list,    
    

/Users/imac086/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
ed = education_details()